# From Stata to SQL
We want to take in the stata code provided by ipums and move it to SQL.

Start by loading the table into SQL with just one column "X1"

In [2]:
import re
import pandas as pd

table_name = "usa16_staging"
f = open("C:\\Users\\brian\\usa16\\usa16_stata.txt", "r")
t_list = list()

x = f.readlines()
for txt in x:
    txt = txt.strip()
    txt = re.split(" +",txt)
    t_list.append(txt)
    
print(t_list[7])

['int', 'year', '1-4', '///']


## Splitting off the variables from X1

In [14]:
stata_data_types = ["int","byte","double","long"]

headers = list()
for i in t_list:
    if i[0] in stata_data_types:
        headers.append(i)
        
def get_data_type (t):
    if t == "int":
        dt = "int"
    elif t == "double":
        dt = "float"
    else:
        dt = "varchar"
    return(dt)

code_list = list()
for i in headers:
    name = i[1]
    start = re.split("-+",i[2])[0]
    end = re.split("-+",i[2])[1]
    dt = get_data_type(i[0])
    c = name + " =  cast(substring(X1," + start + ", " + str(int(end)-int(start)+1) + ") as " + dt + ")"
    code_list.append(c)

code_list[2]


'serial =  cast(substring(X1,7, 8) as float)'

## Creating the definition labels

In [19]:
label_def = list()
for i in t_list:
    if i[0]=="label" and i[1]=="define":
        label_def.append(i)
        
# get columns for table
variable = list()
val = list()
lbl = list()

for i in label_def:
    variable.append(i[2].replace("_lbl",""))
    val.append(i[3])    
    lbl.append(i[4])

sales = [('variable', variable),
         ('val', val),
         ('lbl', lbl)]
    
def_lbl = pd.DataFrame.from_items(sales)

#print(len(variable), len(val), len(lbl))
def_lbl.head()


    
        

,variable,val,lbl
0,year,1850,"`""1850""'"
1,year,1860,"`""1860""',"
2,year,1870,"`""1870""',"
3,year,1880,"`""1880""',"
4,year,1900,"`""1900""',"
